**Dopasowanie modelu do danych** metodą najmniejszych kwadratów (ang. *least-squares method*)

Import danych z internetu działa na takiej samej zasadzie, jak na poprzednim laboratorium.

In [64]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"
resp = requests.get(URL) 
if resp.status_code == 200:
    soup = BeautifulSoup(resp.content,'html.parser')
    tab = soup.find('table',{'class':"wikitable"})
df = pd.read_html(str(tab))[0]
df =df.loc[:, 'City':'Officialpopulation':]
df.replace(to_replace={'[[a]]':"",'[[b]]':""}, regex=True,inplace=True)
df.replace(to_replace='[[,]', value="", regex=True,inplace=True)
df

,City,Country,Officialpopulation
0,Istanbul,Turkey,15840900
1,Moscow,Russia,12632409
2,London,United Kingdom,9002488
3,Saint Petersburg,Russia,5376672
4,Berlin,Germany,3664088
5,Madrid,Spain,3305408
6,Kyiv,Ukraine,2920873
7,Rome,Italy,2844750
8,Bucharest,Romania,2161347
9,Paris,France,2139907


Moim celem będzie sprawdzenie, czy zaludnienie danego kraju wpływa na liczbę mieszkańców w największych miastach. Aby to osiągnąć, ściągam z Wikipedii dane dotyczące ludności w europejskich państwach

In [67]:
import re
URL = "https://en.wikipedia.org/wiki/Area_and_population_of_European_countries"
resp = requests.get(URL) 
if resp.status_code == 200:
    soup = BeautifulSoup(resp.content,'html.parser')
    tab = soup.find('table',{'class':"wikitable"})
df1 = pd.read_html(str(tab))[0]
df1["Name"] = df1['Name'].str.replace(r" \(.*\)","")
pop = dict(zip(df1["Name"],df1["Population"]))
i=0
for country in df["Country"]:
    df.at[i,"Country Population"] = pop.get(country)
    i+=1
df = df.astype({"Country Population": int})
df

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_20608\1113464422.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["Name"] = df1['Name'].str.replace(r" \(.*\)","")


,City,Country,Officialpopulation,Country Population
0,Istanbul,Turkey,15840900,11241000
1,Moscow,Russia,12632409,113112406
2,London,United Kingdom,9002488,66040229
3,Saint Petersburg,Russia,5376672,113112406
4,Berlin,Germany,3664088,82887000
5,Madrid,Spain,3305408,46659302
6,Kyiv,Ukraine,2920873,42220824
7,Rome,Italy,2844750,60404355
8,Bucharest,Romania,2161347,19523621
9,Paris,France,2139907,65167000


Celem ćwiczenia jest utworzenie modelu, muszę zatem podzielić dane na dwa zbiory. 

In [68]:
X = df['Country Population'].values
Y = df['Officialpopulation'].values

print('Ludność kraju: %s' % X)
print('Liczba mieszkańców miasta: %s' % Y)

Ludność kraju: [ 11241000 113112406  66040229 113112406  82887000  46659302  42220824
  60404355  19523621  65167000   9498700   8857960  82887000  38433600
   9778371   7001444  46659302  82887000  42220824  60404355  10625449
   7050034 113112406 113112406 113112406  66040229 113112406 113112406
  82887000 113112406 113112406 113112406]
Liczba mieszkańców miasta: ['15840900' '12632409' '9002488' '5376672' '3664088' '3305408' '2920873'
 '2844750' '2161347' '2139907' '2009786' '1931830' '1852478' '1792718'
 '1723836' '1694480' '1636732' '1488202' '1414319' '1374582' '1335084'
 '1308412' '1257341' '1244254' '1144759' '1140525' '1137704' '1125933'
 '1083498' '1050602' '1049199' '1004763']


In [ ]:
import seaborn as sns
import matplotlib as plt
plt.figure(figsize=(6,6))
sns.scatterplot(
    x = 'Officialpopulation',
    y = 'City population',
    data = df,
    hue = 'City population',
    size = 'Officialpopulation',
    sizes = (20, 200),
    legend = 'brief'
)
plt.xlabel("Ludność kraju", size=16)
plt.ylabel("Populacja miasta", size=16)
plt.title("Porównanie ludności", size=24)
plt.show()